# Steps to deglitch seismic signals

This document is a working document to explain the different steps to deglitch SEIS signal. 

Algorithms mainly come from Philippe Lognonné deglitching code. 
 


## Main architecture of the code

<img src="plots/DeglitchCodeArchitecurev1.0.png">


Preprocessing aspects are not presented here. Lack of time...

## The Green functions

The Philippe's idea was to consider a glitch as step function in acceleration + a step in displacement

$U(t) = \phi_A.H(t-t_0)*T_{ACC}(t)+\phi_D.H(t-t_0)*T_{DIS}(t)$

Where $T_{ACC}$ and $T_{DIS}$ are the seismometer response to a Dirac impulse in acceleration and displacement respectively.

With $T_{DIS}=\ddot{T}_{ACC}$

So we have to apply the instrument response to this Heavyside function and estimate the derivatives
The tranfert function (following plot) is given by the poles and the zeros stored in the dataless files

For more details, see the supplementary material of the article J. Scholz & al (Detection, analysis and removal of glitches from InSight's Seismic data from Mars, Earth and Space Science)

Example for VBB VEL: 

$zero= [0,0,-109.316, -10.1317]$
$pole= [-46.7264+73.9273i, -46.7264-73.9273i, -11.7674+0i, -0.243358+0.30515i, -0.243358-0.30515i]$
$gain= 86262322608$
$A_0  = 80.2887$


$\begin{equation*}
H(s)=k\frac{\Pi_{i=1}^{m}(s-z_i)}{\Pi_{j=1}^{n}(s-p_j)}
\end{equation*}$


Where $k = gain\times A_0$

### Matlab point of view

1. Get the poles, zeros and gains is given by the function `read_resp_v2` 
2. ZPK model is created with `zpk()`

    $\texttt{sys = zpk(uzero_init, upole, ugain*uA0);}$  
    
    
3. Apply this transfer function to an Heavyside is given by the command `step()`
    
    $\texttt{green_20sps= step(sys, time_20sps);}$


### Transfert function of the VBB VEL sensors

<img src="plots/VBB_VEL_Bode.jpg">


### Plot of the Green's function

<img src="plots/Greens_func_and_derivatives.jpg">


These function a made in the function `get_green_func.m`


## Modelisation

### Fit using a single source in acceleration

Can be modelised as follow 
$S(t) = aG(t) + bG'(t) \tag{1}$

We are using the Green's function and it's derivative.


$\begin{equation*}
\begin{pmatrix}
\hat y(t_1)\\
\hat y(t_2)\\
\vdots     \\
\hat y(t_m)
\end{pmatrix} 
=
\begin{pmatrix}
G(t_1) & G'(t_1) \\
G(t_2) & G'(t_2) \\
\vdots  & \vdots \\
G(t_m) & G'(t_m)
\end{pmatrix}
\cdot
\begin{pmatrix}
a \\
b
\end{pmatrix} 
\end{equation*} \tag{2}$

Which has the classical from $\hat y = X.\theta$

It's a least square minimization on a linear system: 

Normal equation: 

$\theta = (X^T.X)^{-1}.X^T.y \tag{3} $

Where $y(t)$ is the original seismic signal. 

The objective function $J(\theta)$ as the following form

$J(\theta) = \sum\limits_{i=1}^{i=m} [\hat{y}(x_i,\theta)-y_i]^2 \tag{4}$

in matrix notation : 

$\begin{align*}
J(\theta) &= (\hat{y}-y)^T(\hat{y}-y)\\
          &=(Xa-y)^T(Xa-y)\\
          &=(a^TX^T-t^T)(Xa-y) \\
          &=a^TX^TXa - a^TX^Ty -y^TXa + y^Ty
\end{align*} \tag{5}$

$\frac{\partial J}{\partial\theta} = 0 \iff X^TXa = X^Ty \tag{6}$


### First improvment : give more weight on the significant part of the signal

<img src="plots/WeightMask.jpg">
The size of the windows is given by two parameters. 
So Let's $W$ be the weight vector.

The normal equation becomes: 
    
$\theta = (X^T.W.X)^{-1}.X^T.W.y \tag{7}$

### Add the step in displacement (called precursor)

Can be modelised as follow $𝑆(𝑡)=𝑎𝐺(𝑡)+𝑏\dot{𝐺}(𝑡)+c\ddot{𝐺}(t)+d\dddot{G}(t)$

$\begin{equation*}
\begin{pmatrix}
\hat y(t_1)\\
\hat y(t_2)\\
\vdots     \\
\hat y(t_m)
\end{pmatrix} 
=
\begin{pmatrix}
G(t_1) & G'(t_1) & \ddot{G}(t_1) & \dddot{G}(t_1)\\
G(t_2) & G'(t_2) & \ddot{G}(t_2) & \dddot{G}(t_2)\\
\vdots  & \vdots & \vdots        & \vdots \\ 
G(t_m) & G'(t_m) & \ddot{G}(t_m) & \dddot{G}(t_m)
\end{pmatrix}
\cdot
\begin{pmatrix}
a \\
b \\
c \\
d
\end{pmatrix} 
\end{equation*} \tag{8}$


### Add a slope under the signal 

This is done using two other parameters and adding two new vectors in the matrix to invert. 
- The first vector $U$ is unit vector with the same length as the Green's functions
- The second vector $S$ is a slope vector with the same length as the Green's functions

So the previous matrix become 

$\begin{equation*}
\begin{pmatrix}
\hat y(t_1)\\
\hat y(t_2)\\
\vdots     \\
\hat y(t_m)
\end{pmatrix} 
=
\begin{pmatrix}
G(t_1) & G'(t_1) & \ddot{G}(t_1) & \dddot{G}(t_1) & U(t_1) & S(t_1)\\
G(t_2) & G'(t_2) & \ddot{G}(t_2) & \dddot{G}(t_2) & U(t_2) & S(t_2)\\
\vdots  & \vdots & \vdots        & \vdots & \vdots & \vdots \\ 
G(t_m) & G'(t_m) & \ddot{G}(t_m) & \dddot{G}(t_m)  & U(t_m) & S(t_m)
\end{pmatrix}
\cdot
\begin{pmatrix}
a \\
b \\
c \\
d \\
e \\
f
\end{pmatrix} 
\end{equation*} \tag{9}$



### Add contraints on the fit

In order to make the fit correct, the first point of the fit and the last point of the fit must be the same as the time series.

In practice, we use the Lagrangian multipliers to make sure that 

$\begin{cases}
\hat{y}(t_1) = y(t_1) \\
\hat{y}(t_m) = y(t_m)
\end{cases} \tag{10}$



The matrix of constraints can be written as 

$\begin{cases}
y(t_1) = \hat{y}(t_1) = a.G(t_1) + b.G'(t_1) + c.\ddot{G}(t_1) + d.\dddot{G}(t_1) + d.U(t_1) + f.S(t_1) \\
y(t_m) = \hat{y}(t_m) = a.G(t_m) + b.G'(t_m) + c.\ddot{G}(t_m) + d.\dddot{G}(t_m) + d.U(t_m) + f.S(t_m)
\end{cases} \tag{11}$

or in the matrix form

$\begin{equation*}
\begin{pmatrix}
y(t_1)\\
y(t_m) 
\end{pmatrix} 
=
\begin{pmatrix}
G(t_1) & G'(t_1) & \ddot{G}(t_1) & \dddot{G}(t_1) & U(t_1) & S(t_1)\\
G(t_m) & G'(t_m) & \ddot{G}(t_m) & \dddot{G}(t_m)  & U(t_m) & S(t_m)
\end{pmatrix}
\cdot
\begin{pmatrix}
a \\
b \\
c \\
d \\
e \\
f
\end{pmatrix} 
\end{equation*} \tag{12}$

Let's write it in short $b=A.\theta$

Now the cost function should be written as 

$J_A(a,\lambda) = \theta^TX^TX\theta - \theta^TX^Ty - y^TX\theta + y^Ty + \lambda^T(A\theta-b) \tag{13}$

After Minimizing $J_A$ with respect to $\theta$ and maximizing $J_A$ with respect to $\lambda$ it comes 

$\begin{equation*}
\begin{pmatrix}
2X^TX & A^T \\
A     & 0 
\end{pmatrix}
\cdot
\begin{pmatrix}
\theta^*\\
\lambda^* 
\end{pmatrix}
=
\begin{pmatrix}
2X^Ty \\
b
\end{pmatrix}
\end{equation*} \tag{14}$

Where $\theta^*$ are the optimum coefficients and $\lambda^*$ the Lagrange multipliers.

For a single glitch: The function is `fit_one_glitch_lagrange.m`


In every case, after finding the coefficients $\theta$, we can estimate the optimal function as 

$\begin{equation*}
\hat{y} = \theta.X
\end{equation*} \tag{15}$


## Identify possible glitches

One way to do it is to first identify the extrema of the input signal. It can be done easely in Matlab using the commands $\texttt{localmax}$ and $\texttt{localmin}$.

The list of extrema is made using the function `find_local_maxima.m`






<img src="plots/FindExtremaOnU_sol82.jpg">
Here is an example on the SOL 82 on axis U

There are several way to fit the extrema 
1. For each axes, fit the extrema of this axis only
2. Create a single list of extrema merging the lists of each axis. Then fit these extrema over the 3 axes.

Since, the goal of this program is to split glitches into families (U only, V only... UVW with tilt... to be precised), we took the second way.


#### Can we fit one glitch after another one ? 

In the following plot, we are looping over the list of extrema. If it works pretty well on the single extrema (no glitch around in a given radius to be discussed), the result is terrible in the cases where glitches are to closed.  

<img src="plots/FitOneGlitchAfterAnotherOne.jpg">


#### So, what can we do ? 

1. Keep the full list and sort the extrema by amplitude ie. fit the biggest extrema first and then fit the smaller ones.
2. Separate the single extrema (easy to fit with the current model) and the clusters of clusters of extrema. 

We decided, not to investigate in the first solution but to go throught the second one.
So, taking a radius parameters, we "extracted" the single extrema and fitted them separately. 

The function to extract the list of single glitches is `get_single_extrema.m`

#### Another weird case : the extrema separated from 1 or 2 bins

Since we decided to merge all the extrema in a single list, it appears that an extrema can be found on an axis at an index $idx$ and an extrema can be found on another axis at some indexes $idx \pm \delta_{idx}$ with $\delta_{idx}$ within $[1,3]$.


<img src="plots/ExtremelyCloseGlitches.jpg">


Our hypothesis is that it's in fact the same glitch existing on two or three axes but, because of the sampling, they are not exactly aligned. 

So we decided to merge those extrema in the list to their barycenter. 

In the example above, we only keep to extrema.

All the single glitches are fitted with the function `fit_list_of_single_gliches` which call for every single glitch on the 3 axes, the function `fit_one_glitch_lagrange`.

Until now, we were dealing with the case where extrema were not mixed. But it's far from the reality. Some time, a second glitch start very quickly after the first one. 

That's why, it's necessary to improve the single extremum fitting model in case we have multiple extrema.

Using the function `clusterize_extrema.m`one can see that there are numerous glitches mixed together. This function take an argument called "radius_max" and if two glitched are closer than this value, they are considered as entangled. In the following figure, $\texttt{radius_max = 36}$ which corresponds to $18 s$ betwwen the two max. Note that the single glitches were already removed from the list.

Most of them are double glitches but we can have more glitched entangled. 
It's important to note that the 


<img src="plots/HistoSizeGlitches.jpg">

#### Modify the model of single extremum fitting to fit more

After clusterizing the extrema, we have to fit then simultaneously.
It's a quiet easy, we just have to change $X$ and $A$ in the equation $15$

1. For N glitches, create your matrix with N blocks with your Green's functions % The two parameters to fit the slope under the whole fraction of signal considered to be fitted. 

2. Take care that all the Green's functions must have the same size, so they must be enlarged on the left and/or on the right. The hypothesis for this is to enlarge with the first point of the function on the left and with the last point on the right. 

Below, an example with the first two blocks of Green's functions.


$\begin{equation*}
\small{
\begin{pmatrix}
G_1(t_1) & G_1'(t_1) & \ddot{G_1}(t_1) & \dddot{G_1}(t_1) & G_2(t_1) & G_2'(t_1) & \ddot{G_2}(t_1) & \dddot{G_2}(t_1)& \dots & U(t_1) & S(t_1)\\
G_1(t_2) & G_1'(t_2) & \ddot{G_1}(t_2) & \dddot{G_1}(t_2) & G_2(t_2) & G_2'(t_2) & \ddot{G_2}(t_2) & \dddot{G_2}(t_2)& \dots & U(t_2) & S(t_2)\\
\vdots  & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots &\vdots & \vdots & \vdots \\ 
G_1(t_m) & G_1'(t_m) & \ddot{G_1}(t_m) & \dddot{G_1}(t_m)  & G_2(t_m) & G_2'(t_m) & \ddot{G_2}(t_m) & \dddot{G_2}(t_m)& \dots & U(t_m) & S(t_m)
\end{pmatrix}}
\end{equation*} \tag{16}$





<img src="plots/IterativeFit_vs_SimultGlitches.jpg">


The plot above is showing a comparison between a deglitching with an iterative way and the simultaneous fit way.

We focus on this 3 glitches on the portion of signal. In the first 3 subplots, on can see the iterative way to fit glitches one by one. After the 3 fits, one can see that the residual is completly wrong. As said previously, we should probably had to fit the glitches according to their amplitude (descending order).

In the last subplot, the program fitted the 3 glitches in the mean time. As we can observe, it's better look visually but better more than all, the variance of the residual is much lower than in the previous case. 


## Fit the glitches in one time or per decreasing amplitude

Dans le cadre de ce programme d'analyse de glitches, les deux types de fit ont été développés:
- Fitter les glitches un par un: On a besoin de cette fonction de toute manière pour les glitches éloignés. On utilise la même pour les glitches plus proches.
- Fitter les glitches proches en une seule passe: cette fonction que je n'utilise pas par la suite, permet de déglitcher le signal. 






## Contraintes de ratio

Le ratio entre le glitch et sa dérivée est appliqué comme contrainte sur le ratio entre le précurseur et sa dérivée. 
Dans ce cas, je prends les glitches par ordre décroissant de d'amplitude.
Et pour chaque glitch, je commence par fitter l'axe sur lequel l'amplitude est la plus importante.

- Un premier fit permet de calculer les coefficients $a$ et $b$ pour les glitchs.
- Un second fit où je vais utiliser le ratio $b/a$ comme ratio entre le précurseur et sa dérivée.




